In [5]:
import re

from notion_client import Client
import os
from dotenv import load_dotenv
from quant.utils import link2id
import pprint
from rich import print as prettyprint
import uuid
load_dotenv()

True

In [2]:
api_key = os.getenv('NOTION_KEY')

def fetch_notion_data(token, id):
    notion = Client(auth=token)
    page_data = notion.blocks.children.list(block_id=id)
    return page_data

In [3]:
link = "https://www.notion.so/ruyoga/testing-9dc1cff1b6e14172ab35cb42a1b4bfda?pvs=4"
id = link2id(link)

page_data = fetch_notion_data(api_key, id)

In [ ]:
def gen_block_id(expression):
    return str(uuid.uuid4())

def parse_text_to_json(text):
    blocks = []
    
    # split string into paragraphs
    paragraphs = text.split('\n\n')
    for paragraph in paragraphs:
        if re.fullmatch(r'\\\[(.*?\\\])', paragraph.strip()):
            blocks.append(create_equation_block(paragraph.strip()[2:-2]))
        else:
            # Split the paragraph into text and inline equations
            parts = re.split(r'(\\\[(.*?)\\\])', paragraph)
            rich_text = []
            for part in parts:
                if part:
                    if re.fullmatch(r'\\\[(.*?)\\\]', part):
                        rich_text.append(create_equation_inline(part[2:-2]))
                    else:
                        rich_text.append(create_text(part))
            if rich_text:
                blocks.append(create_paragraph_block(rich_text))

    return {
        'object': 'list',
        'results': blocks,
        'next_cursor': None,
        'has_more': False,
        'type': 'block',
        'block': {},
        'request_id': generate_block_id()
    }

In [7]:
def create_equation_inline(expression: str):
    """
    :param expression: math expression (katex) 
    :return: json
    """
    return {
        'type': 'equation',
        'equation': {
            'expression': expression
        },
        'annotations': {
            'bold': False,
            'italic': False,
            'strikethrough': False,
            'underline': False,
            'code': False,
            'color': 'default'
        },
        'plain_text': expression,
        'href': None
    }

def create_text(content):
    return {
        'type': 'text',
        'text': {
            'content': content,
            'link': None
        },
        'annotations': {
            'bold': False,
            'italic': False,
            'strikethrough': False,
            'underline': False,
            'code': False,
            'color': 'default'
        },
        'plain_text': content,
        'href': None
    }

def create_equation_block(expression):
    return {
        'object': 'block',
        'id': generate_block_id(),
        'parent': {'type': 'page_id', 'page_id': '9dc1cff1-b6e1-4172-ab35-cb42a1b4bfda'},
        'created_time': '2023-12-08T15:54:00.000Z',
        'last_edited_time': '2023-12-08T15:55:00.000Z',
        'created_by': {'object': 'user', 'id': '97eafb56-9524-418d-a834-2e879d8199fb'},
        'last_edited_by': {'object': 'user', 'id': '97eafb56-9524-418d-a834-2e879d8199fb'},
        'has_children': False,
        'archived': False,
        'type': 'equation',
        'equation': {'expression': expression}
    }

def create_paragraph_block(rich_text):
    return {
        'object': 'block',
        'id': generate_block_id(),
        'parent': {'type': 'page_id', 'page_id': '9dc1cff1-b6e1-4172-ab35-cb42a1b4bfda'},
        'created_time': '2023-12-08T15:54:00.000Z',
        'last_edited_time': '2023-12-08T15:55:00.000Z',
        'created_by': {'object': 'user', 'id': '97eafb56-9524-418d-a834-2e879d8199fb'},
        'last_edited_by': {'object': 'user', 'id': '97eafb56-9524-418d-a834-2e879d8199fb'},
        'has_children': False,
        'archived': False,
        'type': 'paragraph',
        'paragraph': {'rich_text': rich_text, 'color': 'default'}
    }